In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! pip install transformers

     |████████████████████████████████| 1.5MB 11.5MB/s 
     |████████████████████████████████| 890kB 36.0MB/s 
     |████████████████████████████████| 2.9MB 37.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=c5d78288c65206221101cdf6055fafd26c82b02159e313e9baad828b4b97bb61
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
import pandas as pd
import random
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.nn import Parameter
from transformers import BertModel
from transformers import BertConfig
from transformers import BertTokenizer
from keras.preprocessing.sequence import pad_sequences
import torch.optim as optim

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
def get_df(file):
    return pd.read_csv(file,sep = '\t')

In [ ]:
train_df = get_df('/content/drive/My Drive/fake_news_dataset/mediaeval2016/train_posts.txt')
test_df = get_df('/content/drive/My Drive/fake_news_dataset/mediaeval2016/test_posts.txt')

In [ ]:
train_df

,post_id,post_text,user_id,image_id(s),username,timestamp,label
0,324597532548276224,Don't need feds to solve the #bostonbombing wh...,886672620,"boston_fake_03,boston_fake_35",SantaCruzShred,Wed Apr 17 18:57:37 +0000 2013,fake
1,325145334739267584,PIC: Comparison of #Boston suspect Sunil Tripa...,21992286,boston_fake_23,Oscar_Wang,Fri Apr 19 07:14:23 +0000 2013,fake
2,325152091423248385,I'm not completely convinced that it's this Su...,16428755,boston_fake_34,jamwil,Fri Apr 19 07:41:14 +0000 2013,fake
3,324554646976868352,Brutal lo que se puede conseguir en colaboraci...,303138574,"boston_fake_03,boston_fake_35",rubenson80,Wed Apr 17 16:07:12 +0000 2013,fake
4,324315545572896768,4chan and the bombing. just throwing it out th...,180460772,boston_fake_15,Slimlenny,Wed Apr 17 00:17:06 +0000 2013,fake
...,...,...,...,...,...,...,...
15624,578433150071775232,Un présentateur de la ZDF confesse avoir truqu...,257551211,varoufakis_1,Cdt_Sylvestre,Thu Mar 19 05:49:44 +0000 2015,fake
15625,578433646597656576,"Oh les kleine menteurs \""@CorineBarella: Un pr...",27575883,varoufakis_1,damomarc,Thu Mar 19 05:51:42 +0000 2015,fake
15626,578486910491996160,Este es el programa de ZDF en el que confirman...,2049211,varoufakis_1,javierpascual,Thu Mar 19 09:23:21 +0000 2015,fake
15627,578505023912591360,11.34 - wir haben FAST Mittag ▶ Riesen Verwirr...,262222386,varoufakis_1,aotto1968_2,Thu Mar 19 10:35:20 +0000 2015,fake


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
r = 0
f = 0
for x in train_df['label'] :
  if(x=='real') :
    r+=1
  else :
    f+=1

print(r)
print(f)

6225
9404


In [ ]:
x_train = train_df['post_text']
x_test = test_df['post_text']
y_train = train_df['label'].eq('real').astype(int)
y_test = test_df['label'].eq('real').astype(int)

In [ ]:
def get_token_ids(x_train, x_test):
    
    token_tr = []
    token_tst = []
    count = 0
    for sent in x_train :
        tokens = tokenizer.encode(sent, add_special_tokens = True, max_length=512)
        token_tr.append(tokens)
        count+=1
        if(count%1000==0):
            print(count)
    
    for sent1 in x_test :
        tokens1 = tokenizer.encode(sent1, add_special_tokens = True, max_length=512)
        token_tst.append(tokens1)
        count+=1
        if(count%1000==0):
            print(count)
            
    return token_tr, token_tst 

In [ ]:
xtr_token, xtst_token = get_token_ids(x_train, x_test)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000


In [ ]:
xtr_token = pad_sequences(xtr_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")
xtst_token = pad_sequences(xtst_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")

In [ ]:
attention_mask_tr = []
attention_mask_tst = []
for sent in xtr_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tr.append(att_mask)

for sent in xtst_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tst.append(att_mask)

In [ ]:
train_input = torch.tensor(xtr_token)
test_input = torch.tensor(xtst_token)

train_label = torch.tensor(y_train)
test_label = torch.tensor(y_test)

train_mask = torch.tensor(attention_mask_tr)
test_mask = torch.tensor(attention_mask_tst)

In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32

train_data = TensorDataset(train_input, train_mask, train_label)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

In [ ]:
# Create the DataLoader for our validation set.
test_data = TensorDataset(test_input, test_mask, test_label)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
def freeze_layers(model):
    for child in model.children():
        for param in child.parameters():
            param.requires_grad = False

In [ ]:
class BertMapping(nn.Module):
    """
    """
    def __init__(self):
        super(BertMapping, self).__init__()
        # bert_config = BertConfig.from_json_file(opt.bert_config_file)
        bert_config = BertConfig.from_pretrained('bert-base-uncased')
        self.bert = BertModel(bert_config)
        # self.bert.load_state_dict(torch.load(opt.init_checkpoint, map_location='cpu'))
        freeze_layers(self.bert)
        final_dims = 256
        # self.txt_stru = opt.txt_stru

        # if opt.txt_stru == 'pooling':
        #     self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        #     self.mapping = nn.Linear(bert_config.hidden_size, opt.final_dims)
        # elif opt.txt_stru == 'cnn':
        Ks = [1, 2, 3]
        in_channel = 1
        out_channel = 512
        embedding_dim = bert_config.hidden_size
        self.convs1 = nn.ModuleList([nn.Conv2d(in_channel, out_channel, (K, embedding_dim)) for K in Ks])
        self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        self.mapping = nn.Linear(len(Ks)*out_channel, final_dims)
        self.cls_layer = nn.Linear(final_dims, 1)
        # elif opt.txt_stru == 'rnn':
        #     embedding_dim = bert_config.hidden_size
        #     self.bi_gru = opt.bi_gru
        #     self.rnn = nn.GRU(embedding_dim, opt.embed_size, opt.num_layers, batch_first=True, bidirectional=opt.bi_gru)
        #     self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        #     self.mapping = nn.Linear(opt.embed_size, opt.final_dims)
        # elif opt.txt_stru == 'trans':
        #     bert_config = BertConfig.from_json_file(opt.img_trans_cfg)
        #     self.layer = bert.BERTLayer(bert_config)
        #     self.dropout = nn.Dropout(bert_config.hidden_dropout_prob)
        #     self.mapping = nn.Linear(768, opt.final_dims)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids,attention_mask=attention_mask, return_dict=True)
        # if self.txt_stru == 'pooling':
        #     output = self.mapping(all_encoder_layers[-1])
        #     output = torch.mean(output, 1)
        #     code = output
        # elif self.txt_stru == 'cnn':
        x = outputs.last_hidden_state.unsqueeze(1)  # (batch_size, 1, token_num, embedding_dim)
        x = [F.relu(conv(x)).squeeze(3) for conv in self.convs1]  # [(batch_size, out_channel, W), ...]*len(Ks)
        x = [F.max_pool1d(i, i.size(2)).squeeze(2) for i in x]  # [(N, Co), ...]*len(Ks)
        output = torch.cat(x, 1)
        # elif self.txt_stru == 'rnn':
        #     x = all_encoder_layers[-1]  # (batch_size, token_num, embedding_dim)
        #     packed = pack_padded_sequence(x, lengths, batch_first=True)
        #     # Forward propagate RNN
        #     out, _ = self.rnn(packed)
        #     # Reshape *final* output to (batch_size, hidden_size)
        #     padded = pad_packed_sequence(out, batch_first=True)
        #     cap_emb, cap_len = padded
        #     if self.bi_gru:
        #         cap_emb = (cap_emb[:, :, :cap_emb.size(2) / 2] + cap_emb[:, :, cap_emb.size(2) / 2:]) / 2
        #     else:
        #         cap_emb = cap_emb
        #     output = torch.mean(cap_emb, 1)
        # elif self.txt_stru == 'trans':

        #     hidden_states = self.mapping(all_encoder_layers[-1])
        #     extended_attention_mask = attention_mask.unsqueeze(1).unsqueeze(2)
        #     extended_attention_mask = extended_attention_mask.float()
        #     extended_attention_mask = (1.0 - extended_attention_mask) * -10000.0
        #     hidden_states = self.layer(hidden_states, extended_attention_mask)
        #     # output = hidden_states[:, 0, :]
        #     output = torch.mean(hidden_states, 1)

        output = self.dropout(output)
        code = self.mapping(output)
        # code = F.tanh(code)
        code = F.normalize(code, p=2, dim=1)
        code = self.cls_layer(code)
        return code

In [ ]:
def get_accuracy_from_logits(logits, labels):
    probs = torch.sigmoid(logits.unsqueeze(-1))
    soft_probs = (probs > 0.5).long()
    acc = (soft_probs.squeeze() == labels).float().mean()
    return acc

In [ ]:
def train(net, criterion, opti, train_loader, num_epochs, val_loader):
  for epoch in range(num_epochs):
    loss_val = 0
    for it, (seq, attn_masks, labels) in enumerate(train_loader):
        #Clear gradients
        opti.zero_grad()  
        #Converting these to cuda tensors
        seq, attn_masks, labels = seq.cuda(), attn_masks.cuda(), labels.cuda()

        #Obtaining the logits from the model
        logits = net(seq, attn_masks)

        #Computing loss
        loss = criterion(logits.squeeze(-1), labels.float())
        loss_val += loss.data

        #Backpropagating the gradients
        loss.backward()

        #Optimization step
        opti.step()

        if (it + 1) % 20 == 0:
            acc = get_accuracy_from_logits(logits, labels)
            print("Iteration {} of epoch {} complete. Loss : {} Accuracy : {}".format(it+1, epoch+1, loss.item(), acc))

        del seq
        del attn_masks
        del labels
        del logits
        del loss
        torch.cuda.empty_cache()

    print('Epoch [{}/{}], Loss:{:.4f}'.format(epoch+1, num_epochs, loss_val))
    # if((epoch+1)%2==0) :
    #   print("classification_report")
    #   print(eval_model(net, val_loader))
    # print("--------------------------------------------------------------")

In [ ]:
net = BertMapping().to(device)
criterion = nn.BCEWithLogitsLoss()
opti = optim.Adam(net.parameters(), lr = 2e-5)

In [ ]:
train(net, criterion, opti, train_dataloader, 20, test_dataloader)

Iteration 20 of epoch 1 complete. Loss : 0.6427617073059082 Accuracy : 0.6875
Iteration 40 of epoch 1 complete. Loss : 0.7217259407043457 Accuracy : 0.46875
Iteration 60 of epoch 1 complete. Loss : 0.6871233582496643 Accuracy : 0.5625
Iteration 80 of epoch 1 complete. Loss : 0.650540292263031 Accuracy : 0.65625
Iteration 100 of epoch 1 complete. Loss : 0.588671088218689 Accuracy : 0.8125
Iteration 120 of epoch 1 complete. Loss : 0.7006068229675293 Accuracy : 0.53125
Iteration 140 of epoch 1 complete. Loss : 0.6626191139221191 Accuracy : 0.625
Iteration 160 of epoch 1 complete. Loss : 0.6868983507156372 Accuracy : 0.5625
Iteration 180 of epoch 1 complete. Loss : 0.6990219354629517 Accuracy : 0.53125
Iteration 200 of epoch 1 complete. Loss : 0.6481145620346069 Accuracy : 0.65625
Iteration 220 of epoch 1 complete. Loss : 0.6750702261924744 Accuracy : 0.59375
Iteration 240 of epoch 1 complete. Loss : 0.6965141296386719 Accuracy : 0.53125
Iteration 260 of epoch 1 complete. Loss : 0.71897584

In [ ]:
def get_labels_from_logits(logits) :
  probs = torch.sigmoid(logits.unsqueeze(-1))
  soft_probs = (probs > 0.5).long()
  labels = soft_probs.squeeze()
  return labels

In [ ]:
def eval_model(model, val_loader) :
  final_out = []
  final_lab = []

  for idx, (val_input, val_mask,val_label) in enumerate(val_loader):
    try:
      val_input = val_input.cuda()
      val_mask = val_mask.cuda()
      output = model(val_input, val_mask)
      output = get_labels_from_logits(output)
      output = output.cpu().detach().numpy()
      val_label = val_label.cpu().detach().numpy()

      final_out.extend(list(output))
      final_lab.extend(list(val_label))

      del val_input
      del val_label
      del val_mask
      del output
      torch.cuda.empty_cache()
    except:
      print("error") 

        
  return classification_report(final_lab, final_out)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
print("classification_report")
print(eval_model(net, test_dataloader))
print("--------------------------------------------------------------")

classification_report
error
              precision    recall  f1-score   support

           0       0.56      0.90      0.69      1185
           1       0.56      0.14      0.23       991

    accuracy                           0.56      2176
   macro avg       0.56      0.52      0.46      2176
weighted avg       0.56      0.56      0.48      2176

--------------------------------------------------------------


In [ ]:
torch.save(net.state_dict(), "/content/drive/My Drive/multi-modal/text_embed_dict.pt")